In [1]:
from tfob import TFOb, BHSA, DSS

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


In [2]:
import pandas as pd
from itertools import chain

In [3]:
motion_verbs = ["BW>[", "HLK[", "CWB[", "<LH[", "CLX[", "JY>[", 
                "JRD[", "<BR[", "NGC[", "QRB[", "NWS[", "CLK[", 
                ">SP[", "NPL[", "QBY[", "LQX[", "SWR[", "GLH[",
                "QHL["]

In [4]:
verbs_bhsa = TFOb.all("word", BHSA).filter(book="Isaiah").filter_in(lex=motion_verbs)
verbs_bhsa

<word_571 "TB>W HBJ> HSJRW LKW JBW> >CJBH >SJRH >CJBH CBJH HLKW LKW N<LH NLKH TY> LKW NLKH BW> B>W JCLJK BW> [...] JRDT HLKJM QRB TGC HWY>TJ T<LJNH M<LH >BJ> JBW> JBW> HCJB B>H QBY B>W CLXTJ HBJ>W JBJ>W >QX JBW> JY>W">

In [5]:
verbs_dss = TFOb.all("scroll", DSS).filter(scroll="1Qisaa").to_words.filter(book="Isaiah").filter_in(lex=motion_verbs)
verbs_dss

<word_577 "TB>W HBJ> HSJRW LKW JBW> HCJB >SJR >CJBH CBJH HLKW LKW N<LH N>LKH TY> LKW NLKH B>W JCLJK BW> MHSJR [...] JRDTH HWLKJM QRB HWYJTJ T<LJN> M<LH >BJ> JBW> JBW> HCJB JBW> B>W QBY B>W CLXTJ HBJ>W JBJ>W >QX JBW> JY>W">

In [6]:
# Notes
# one line per complement
# manage the verses where motion verbs occur several times
# remove underscore in proper name in BHSA and ‘ for pronominal suffixes in DSS

In [7]:
def clean(g_cons):
    return g_cons.replace("_", " ").replace("'", "")  

In [47]:
items = []

for verb in chain(verbs_bhsa, verbs_dss):
    if verb.source.name == "BHSA":
        scroll = "MT"
        verse = verb.to_verses
        dir_he_dss_verse = ""
    else:
        scroll = verb.to_scrolls.scroll[0]
        verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll) 
        dir_he_dss_verse = int("H" in verse.uvf_etcbc)
             
    complements = verb.to_clauses.to_phrases.filter(function="Cmpl")
    if len(complements) == 0:
        complements = [""]
        
    for complement in complements:
        if complement == "":
            dir_he = ""
        else: 
            dir_he = int("H" in complement.to_words.uvf)

        item = {
            "verb_id": verb.ids[0], 
            "lex": verb.lex[0], 
            "scroll": scroll,
            "book": verb.book[0], 
            "chapter": verb.chapter[0], 
            "verse": verb.verse[0],
            "g_cons": clean(verb.g_cons[0]),
            "hebrew": verb.text,
            "g_cons_verse": clean(str(verse)),
            "stem": verb.vs[0],
            "tense": verb.vt[0],
            "subject": clean(str(verb.to_clauses.to_phrases.filter(function="Subj"))),
            "subj_heb": verb.to_clauses.to_phrases.filter(function="Subj").text,
            "complement": clean(str(complement)),
            "cmpl_heb" : verb.to_clauses.to_phrases.filter(function="Cmpl").text,
            "dir_he": dir_he,
            "dir_he_dss_verse": dir_he_dss_verse, 
        }
        if complement != "":
            prepositions = complement.to_words.filter(sp="prep")
            n = 0
            for preposition in prepositions:
                n += 1
                item[f"preposition_{n}"] = str(preposition)
        items.append(item)
           
df = pd.DataFrame(items).fillna("")
df.sort_values(["book", "chapter", "verse", "lex", "scroll"], ascending=[True, True, True, True, False], ignore_index=True, inplace=True) 


cmpl_isaiah = df

In [48]:
cmpl_isaiah.to_csv("mv_cmpl_isaiah_bhsa_dss.csv", index=False, encoding="utf8")

In [52]:
# Show the occurrences of the directive he in the BHSA 

cmpl_isaiah[cmpl_isaiah.dir_he == 1][['lex', "book", "chapter", "verse", "scroll", "hebrew", "complement", 'dir_he']]

,lex,book,chapter,verse,scroll,hebrew,complement,dir_he
125,CLX[,Isaiah,16,1,MT,שִׁלְחוּ־,MDBRH >L HR BT YJWN,1
185,BW>[,Isaiah,20,1,MT,בֹּ֤א,>CDWDH,1
237,<BR[,Isaiah,23,6,MT,עִבְר֖וּ,TRCJCH,1
450,CLX[,Isaiah,36,2,MT,יִּשְׁלַ֣ח,JRWCLMH,1
631,CLX[,Isaiah,43,14,MT,שִׁלַּ֣חְתִּי,BBLH,1
881,CWB[,Isaiah,55,10,MT,יָשׁ֔וּב,CMH,1
1111,BW>[,Isaiah,7,24,MT,יָ֣בֹוא,CMH,1
1113,BW>[,Isaiah,7,25,MT,תָבֹ֣וא,CMH,1


In [53]:
# Show the occurrences of the directive he in the verse where the verb appears in the DSS

cmpl_isaiah[cmpl_isaiah.dir_he_dss_verse == 1][['lex', "book", "chapter", "verse", "scroll", "hebrew", "dir_he_dss_verse"]]

,lex,book,chapter,verse,scroll,hebrew,dir_he_dss_verse
35,<BR[,Isaiah,10,28,1Qisaa,עבר,1
37,BW>[,Isaiah,10,28,1Qisaa,בא,1
91,<LH[,Isaiah,14,13,1Qisaa,אעלה,1
126,CLX[,Isaiah,16,1,1Qisaa,שלחו,1
161,BW>[,Isaiah,19,23,1Qisaa,בא,1
186,BW>[,Isaiah,20,1,1Qisaa,בא,1
188,CLX[,Isaiah,20,1,1Qisaa,שלח,1
238,<BR[,Isaiah,23,6,1Qisaa,עוברי,1
358,JRD[,Isaiah,31,1,1Qisaa,יורדים,1
402,QBY[,Isaiah,34,15,1Qisaa,נקבצו,1


In [24]:
# for name in dir(DSS.F):
#     if "__" in name:
#         continue
#     print(name, getattr(BHSA.F, name).v(verbs.ids[0]))

In [54]:
# Show the occurrences of the directive he (warning: in DSS, we look through the whole verse here, not just the complement)

dh_isaiah["dir_he"] = dh_isaiah.apply(lambda row: row.dir_he or row.dir_he_dss_verse, axis=1)
# del dh_isaiah["dir_he_dss_verse"]
# dh_isaiah
dh_isaiah.style.set_caption("Directive he in verse (DSS) or motion verb complements (BHSA) in Isaiah")

,lex,book,chapter,verse,scroll,hebrew,complement,dir_he
35,<BR[,Isaiah,10,28,1Qisaa,עבר,,1
37,BW>[,Isaiah,10,28,1Qisaa,בא,,1
91,<LH[,Isaiah,14,13,1Qisaa,אעלה,,1
125,CLX[,Isaiah,16,1,MT,שִׁלְחוּ־,MDBRH >L HR BT YJWN,1
126,CLX[,Isaiah,16,1,1Qisaa,שלחו,,1
161,BW>[,Isaiah,19,23,1Qisaa,בא,,1
185,BW>[,Isaiah,20,1,MT,בֹּ֤א,>CDWDH,1
186,BW>[,Isaiah,20,1,1Qisaa,בא,,1
188,CLX[,Isaiah,20,1,1Qisaa,שלח,,1
237,<BR[,Isaiah,23,6,MT,עִבְר֖וּ,TRCJCH,1


In [16]:
verbs_dss._levels

['to_scrolls',
 'to_lexs',
 'to_fragments',
 'to_lines',
 'to_clauses',
 'to_clusters',
 'to_phrases',
 'to_words',
 'to_signs']

In [14]:
TFOb.all("scroll", DSS).filter(scroll="1Qisaa").to_words.filter(book="Isaiah", chapter="47", verse="8").text

'ועתה שמעי זואת עוד׳נה היושבת לבטח האומרה בלבב׳ה אני ואפס׳י עוד לוא אשב עלמנה ולוא אראה שכול ׃ '

In [17]:
TFOb.all("word", BHSA).filter(book="Isaiah", chapter="38", verse="17").text

'הִנֵּ֥ה לְשָׁלֹ֖ום מַר־לִ֣י מָ֑ר וְאַתָּ֞ה חָשַׁ֤קְתָּ נַפְשִׁי֙ מִשַּׁ֣חַת בְּלִ֔י כִּ֥י הִשְׁלַ֛כְתָּ אַחֲרֵ֥י גֵוְךָ֖ כָּל־חֲטָאָֽי׃ '